Import Libraries

In [33]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter
from PIL import Image
from matplotlib.pyplot import specgram
from matplotlib.pyplot import subplots
from matplotlib.mlab import window_hanning
import matplotlib.pyplot as plt
import scipy
import cv2
%reload_ext autoreload
%autoreload 2
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from sklearn import metrics

Import data

In [35]:
# data file
dEDF = pd.read_csv('data.csv').values
# time file
tEDF = pd.read_csv('time.csv').values

In [36]:
max(tEDF)

array([100.])

In [37]:
Fs = 1/(tEDF[3]-tEDF[2])
Fs = Fs[0]
winSize = int(np.round(Fs))
winStep = int(np.round(Fs/2))
movingwin    = [0.200,0.005] # Window size, Step size [s]
iWin   = int(np.round(movingwin[0]*Fs/2)*2);
iStep  = int(np.round(movingwin[1]*Fs/2)*2);
noverlap = iWin-iStep
i_stop = 0
i = 0
counter = 0

In [38]:
def hannspecgramc(d1,movingwin,Fs,tEDF,flo,fhi):
    iStart = 1;
    iWin   = int(np.round(movingwin[0]*Fs/2)*2);
    iStep  = int(np.round(movingwin[1]*Fs/2)*2);
    T      = iWin/Fs;
    df     = 1/T;
    fNQ    = Fs/2;
    freq   = range(0,int(iWin/2+1))*df;
    counter=1;
    S      = np.zeros([int(np.ceil(len(tEDF)/iStep)), int(iWin/2+1)]);
    times  = np.zeros([int(np.ceil(len(tEDF)/iStep)), 1]);
    while iStart+iWin < len(tEDF):
        dnow    = d1[iStart:iStart+iWin-1];
        dnow    = dnow - np.mean(dnow);
        dnow    = np.hanning(len(dnow))*np.squeeze(dnow);
        spectrum= np.real(scipy.fft.fft(dnow)*np.conjugate(scipy.fft.fft(dnow)));
        S[counter,:]   = spectrum[:int(iWin/2+1)];
        times[counter] = tEDF[int(iStart+iWin/2)];

        counter=counter+1;
        iStart = iStart + iStep;
    S = S[:counter-1,:];
    times = times[:counter-1];
    S = S[:,(freq>=flo)&(freq<=fhi)]
    freq = freq[(freq>=flo)&(freq<=fhi)]
    return S,times,freq

In [39]:
start_times = []
stop_times = []

In [40]:
image_no_start_stop_dict = {}

In [41]:
while tEDF[i_stop] < 99: 
#only looking at the first K seconds, can change to any value

    i_start = i
    i_stop  = i_start + winSize - 1

    dspec = dEDF[i_start:i_stop]
    dspec = dspec - np.mean(dspec)
    t     = tEDF[i_start:i_stop]
    flo = 30
    fhi = 250
    movingwin    = [0.200,0.005] # Window size, Step size [s]
    S0,S_times,S_freq = hannspecgramc(dspec,movingwin,Fs,t,flo,fhi)
    #Smooth the spectra.
    t_smooth = 11
    dt_S     = S_times[2]-S_times[1]
    S_smooth = gaussian_filter(S0,1)

    A = np.log10(S_smooth)
    A = np.flipud(A)
    
    #fig, ax = subplots(figsize=(6,6))
    #ax.imshow(A, cmap='jet',
    #       extent = [min(S_freq),max(S_freq),min(S_times)[0],max(S_times)[0]],
    #         aspect = 'auto')
    
    name = 'demo_data/test/img_'+ str(counter)+'.jpg'
    plt.imsave(name,A,cmap='jet')
    
    start_times.append(tEDF[i_start])
    stop_times.append(tEDF[i_stop])
    start_stop = [tEDF[i_start],tEDF[i_stop]]
    
    new_name = 'test/img_'+ str(counter)+'.jpg'
    image_no_start_stop_dict[new_name] = start_stop
    
    i = i_start + winStep;
    counter=counter+1;

In [42]:
# where the data is
PATH = "demo_data"
# using resnet architecture
arch = resnet34
# size of square image in pixels
sz = 44
# transforms used on training data
transforms_up_down = [RandomScale(sz,1.2),RandomRotate(1)]
tfms = tfms_from_model(arch,sz,crop_type = CropType.NO,aug_tfms=transforms_up_down)
# data: comes from PATH, used tfms on training data, bs of 8 for training data, test data located in test folder
data = ImageClassifierData.from_paths(PATH,tfms=tfms,bs=8,test_name='test')
# load in pretraine
state = torch.load('saved_model.pkl',map_location=torch.device('cpu')) # remove map_location parameter if on GPU
learn2 = ConvLearner.pretrained(arch,data,precompute=False)
learn2.model.load_state_dict(state)

<All keys matched successfully>

In [43]:
log_preds_test = learn2.predict(is_test=True)
preds_test = np.argmax(log_preds_test,axis=1)
probs_test = np.exp(log_preds_test[:,1])

In [44]:
test_names = np.empty_like(data.test_ds.fnames)
for i in range(len(data.test_ds.fnames)):
    test_names[i] = data.test_ds.fnames[i]
    #temp = data.test_ds.fnames[i]
    #matchobj = re.search('.*im.*',temp)
    #test_names[i] = matchobj.group()
test_df = pd.DataFrame(data = test_names,columns = ['image_number'])
test_df['prediction'] = preds_test
test_df['probability'] = probs_test
test_df['times'] = test_df['image_number'].map(image_no_start_stop_dict)
#test_df['start time'] = start_times
#test_df['stop time'] = stop_times

In [45]:
image_no_start_stop_dict

{'test/img_0.jpg': [array([0.00098]), array([1.00049])],
 'test/img_1.jpg': [array([0.50074]), array([1.50025])],
 'test/img_2.jpg': [array([1.00049]), array([2.])],
 'test/img_3.jpg': [array([1.50025]), array([2.49975])],
 'test/img_4.jpg': [array([2.]), array([2.99951])],
 'test/img_5.jpg': [array([2.49975]), array([3.49926])],
 'test/img_6.jpg': [array([2.99951]), array([3.99902])],
 'test/img_7.jpg': [array([3.49926]), array([4.49877])],
 'test/img_8.jpg': [array([3.99902]), array([4.99853])],
 'test/img_9.jpg': [array([4.49877]), array([5.49828])],
 'test/img_10.jpg': [array([4.99853]), array([5.99803])],
 'test/img_11.jpg': [array([5.49828]), array([6.49779])],
 'test/img_12.jpg': [array([5.99803]), array([6.99754])],
 'test/img_13.jpg': [array([6.49779]), array([7.4973])],
 'test/img_14.jpg': [array([6.99754]), array([7.99705])],
 'test/img_15.jpg': [array([7.4973]), array([8.49681])],
 'test/img_16.jpg': [array([7.99705]), array([8.99656])],
 'test/img_17.jpg': [array([8.49681]

In [46]:
test_df[test_df['prediction']==1].sort_values('probability')

,image_number,prediction,probability,times
136,test/img_41.jpg,1,0.625198,"[[20.4909090909091], [21.4904176904177]]"
93,test/img_196.jpg,1,0.675399,"[[97.9528255528256], [98.9523341523341]]"
84,test/img_77.jpg,1,0.753126,"[[38.4820638820639], [39.4815724815725]]"
69,test/img_173.jpg,1,0.774374,"[[86.4584766584767], [87.4579852579853]]"
186,test/img_131.jpg,1,0.841441,"[[65.4687960687961], [66.4683046683047]]"
129,test/img_184.jpg,1,0.899287,"[[91.9557739557739], [92.9552825552826]]"
139,test/img_82.jpg,1,0.918211,"[[40.9808353808354], [41.980343980344]]"
180,test/img_23.jpg,1,0.920939,"[[11.4953316953317], [12.4948402948403]]"
37,test/img_113.jpg,1,0.928052,"[[56.4732186732187], [57.4727272727273]]"
48,test/img_149.jpg,1,0.934602,"[[74.4643734643735], [75.4638820638821]]"
